In [173]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import math 
from tensorflow import keras

from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


Uv-Model Testing

In [184]:
UvModel = tf.keras.models.load_model('/gdrive/MyDrive/saved_model/UvRadiation2')
# Check its architecture
UvModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, None, 60)          360       
                                                                 
 lstm (LSTM)                 (None, None, 64)          32000     
                                                                 
 lstm_1 (LSTM)               (None, None, 128)         98816     
                                                                 
 dense (Dense)               (None, None, 60)          7740      
                                                                 
 dropout (Dropout)           (None, None, 60)          0         
                                                                 
 dense_1 (Dense)             (None, None, 10)          610       
                                                                 
 dense_2 (Dense)             (None, None, 1)           1

In [185]:
cobacoba3 = np.random.randint(20,150,27) #random numbers for forecasting uv 

In [186]:
#predicting UV
window_size=24
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

def cvrttolist (listy) : 
  return listy.tolist()
def extenddf(df,forecast):
  a= cvrttolist(df)
  b=cvrttolist(forecast)
  a.extend(b)
  return np.array(a) 


def forecastuv (df, hour):
  for nextuv in range(hour) :
    lstm_forecast2 = model_forecast(UvModel, df[..., np.newaxis], window_size)
    lstm_forecast2 = lstm_forecast2[((len(df)-1) - window_size):-1, -1, 0]
    df=extenddf(df,lstm_forecast2)

    lstm_forecast2 = model_forecast(UvModel, df[..., np.newaxis], window_size)
    lstm_forecast2 = lstm_forecast2[((len(df))-1 - window_size):-1, -1, 0]
    df = extenddf(df,lstm_forecast2)

    df=cvrttolist(df)
    a=df[-2]
    df.remove(a)
    df=np.array(df)

  return cvrttolist(df[-hour:])

def maketotableuv(thelist):
  frames = {  "Hour Forecast":list(range(1,(len(thelist)) +1 ))  ,"UV-Forecast" :thelist   }
  dfforecast = pd.DataFrame(frames)
  return (dfforecast)


maketotableuv(forecastuv(cobacoba3,4))


,Hour Forecast,UV-Forecast
0,1,58.417686
1,2,31.794151
2,3,9.958142
3,4,21.467728


### TEMPRATURE Model Testing

In [177]:
Temprature = tf.keras.models.load_model('/gdrive/MyDrive/saved_model/Temprature')
# Check its architecture
Temprature.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, None, 60)          360       
                                                                 
 lstm (LSTM)                 (None, None, 64)          32000     
                                                                 
 lstm_1 (LSTM)               (None, None, 64)          33024     
                                                                 
 dense (Dense)               (None, None, 30)          1950      
                                                                 
 dense_1 (Dense)             (None, None, 10)          310       
                                                                 
 dense_2 (Dense)             (None, None, 1)           11        
                                                                 
 lambda (Lambda)             (None, None, 1)           0

In [178]:
cobacoba2 = np.random.randint(25,35,70) #random trial dataset for temprature
cobacoba2

array([30, 27, 26, 27, 28, 33, 33, 27, 26, 26, 25, 30, 31, 32, 28, 31, 27,
       28, 26, 33, 33, 28, 32, 26, 31, 34, 30, 29, 30, 30, 26, 33, 29, 33,
       33, 30, 26, 31, 28, 30, 26, 29, 29, 34, 30, 29, 33, 27, 27, 30, 34,
       33, 26, 28, 25, 29, 27, 29, 31, 33, 33, 27, 28, 27, 26, 26, 29, 32,
       32, 27])

In [179]:
def cvrttolist (listy) : 
  return listy.tolist()
def extenddf(df,forecast):
  a= cvrttolist(df)
  b=cvrttolist(forecast)
  a.extend(b)
  return np.array(a) 
  
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

    
def forecasttemprature (df, hour):
  temp = [] 
  for nexttemprature in range(hour) :
    lstm_forecast2 = model_forecast(Temprature, df[..., np.newaxis], window_size)
    lstm_forecast2 = lstm_forecast2[((len(df)-1) - window_size):-1, -1, 0]
    df=extenddf(df,lstm_forecast2)
  
    lstm_forecast2 = model_forecast(Temprature, df[..., np.newaxis], window_size)
    lstm_forecast2 = lstm_forecast2[((len(df)-1) - window_size):-1, -1, 0]
    df=extenddf(df,lstm_forecast2)

    df=cvrttolist(df)
    a=df[-2]
    df.remove(a)
    df=np.array(df)

  return cvrttolist(df[-hour:])

def maketotabletemprature(thelist):
  frames = {  "Hour Forecast":list(range(1,(len(thelist)) +1 ))  ,"Temprature-Forecast" :thelist   }
  dfforecast = pd.DataFrame(frames)
 
  return (dfforecast)


  

In [180]:
temporarydataframe = maketotabletemprature( forecasttemprature(cobacoba2,24))